In [3]:
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
import time

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, TFBertModel

#import tensorflow_hub as hub

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
df1 = pd.read_csv('All_US_tweets.csv')

In [4]:
df1.head()

,Timestamp,TweetText,Closest_State,Closest_City,Region
0,10229,i gotta get you readded to bbm,Alabama,Adamsville,South
1,16439,ahhh yes,Alabama,Adamsville,South
2,17131,an old locksmith,Alabama,Adamsville,South
3,2648,rt they are shooting at pentagon metro please...,Alabama,Adamsville,South
4,2750,as a matter of fact i wanna ask about that,Alabama,Adamsville,South


In [5]:
# Max string length
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    374294.000000
mean         11.843054
std           7.576504
min           1.000000
1%            2.000000
50%          11.000000
99%          29.000000
max         710.000000
Name: TweetText, dtype: float64

In [6]:
max_length = 29

In [7]:
df1['TweetText'] = df1['TweetText'].astype(str)
df1 = df1[df1['TweetText'].str.split(" ").str.len() <= 29]
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    372389.000000
mean         11.709132
std           6.917127
min           1.000000
1%            2.000000
50%          10.000000
99%          28.000000
max          29.000000
Name: TweetText, dtype: float64

In [8]:
# Bert tools

b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def tokenize_tweet(tweet):
    return b_tokenizer(tweet, padding='max_length', truncation=True, return_tensors="tf")

#tokenized_tweets = df['tweet_text'].apply(tokenize_tweet)

def get_bert_embeddings(tokenized_tweet):
    outputs = bert_model(tokenized_tweet)
    return outputs.last_hidden_state[:, 0, :]

#embeddings = tokenized_tweets.apply(get_bert_embeddings)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [9]:
df2 = df1.copy()

In [10]:
df2['TweetText'] = df2['TweetText'].apply(tokenize_tweet)

In [11]:
start = time.time()
df2['TweetText'] = df2['TweetText'].apply(get_bert_embeddings)
end = time.time()

KeyboardInterrupt: 

In [ ]:
total_time = end - start
hours = total_time // 3600
mins = total_time % 3600 // 60
sec = total_time % 3600 % 60

print("Execution took: " + str(hours) + ' hrs, ' + str(mins) + ' min, ' + str(np.round(sec,2)) + 'secs')

In [ ]:
df2.head()

In [ ]:
# one hot encoding
df3 = df2.copy()

one_hot = pd.get_dummies(df3['Region'])
df3 = df3.join(one_hot)
df3 = df3.drop('Region', axis = 1)
df3

In [ ]:
x_dat = df3[['Timestamp','TweetText']]
y_dat = df3[df2['Region'].unique()]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size = 0.3, random_state = 50)

In [ ]:
text_input = Input(shape=(768,), name='text_input')  # BERT base model outputs 768-dimensional embeddings
seconds_input = Input(shape=(1,), name='seconds_input')

combined_input = concatenate([text_input, seconds_input])

dense_layer_1 = Dense(128, activation='relu')(combined_input)
output_layer = Dense(1, activation='sigmoid')(dense_layer_1)

model = Model(inputs=[text_input, seconds_input], outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
target = y_train
seconds = x_train['Timestamp']


In [ ]:
history = model.fit([embeddings, seconds], labels, epochs = 5, batch_size = 32)